---
layout: post
title: Poker
description:  A card game buil to entertain
type: issues 
courses: { csse: {week: 4} }
---

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Poker Game</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #2c3e50;
            color: #ecf0f1;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
        }
        .container {
            max-width: 600px;
            text-align: center;
            background-color: #34495e;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.3);
        }
        h1 {
            font-size: 2rem;
            margin-bottom: 10px;
        }
        label, .result, .error {
            display: block;
            margin: 10px 0;
        }
        input, button {
            padding: 10px;
            font-size: 16px;
            margin-top: 10px;
        }
        .hand {
            display: flex;
            justify-content: center;
            gap: 10px;
            margin-top: 10px;
        }
        .card {
            width: 50px;
            height: 70px;
            border-radius: 5px;
            box-shadow: 0px 2px 5px rgba(0, 0, 0, 0.2);
            display: flex;
            align-items: center;
            justify-content: center;
            font-size: 1.2rem;
            color: #333;
            background-color: #fff;
            position: relative;
        }
        .card.red {
            color: #e74c3c;
        }
        .dropdown {
            margin: 20px 0;
            text-align: left;
        }
        .dropdown-content {
            display: none;
            background-color: #f1f1f1;
            border-radius: 5px;
            padding: 10px;
            box-shadow: 0px 2px 5px rgba(0, 0, 0, 0.2);
            font-size: 0.9rem;
            color: #333;
        }
        .dropdown-content p {
            margin: 5px 0;
        }
        .dropdown:hover .dropdown-content {
            display: block;
        }
        .result, .error {
            margin-top: 15px;
            font-size: 1.1rem;
        }
        .error {
            color: #e74c3c;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>One-Hand Poker Game</h1>
        <label for="betAmount">Enter your bet:</label>
        <input type="number" id="betAmount" min="1" required>
        <button onclick="playGame()">Play</button>
        <div class="dropdown">
            <button>Show Poker Hands</button>
            <div class="dropdown-content">
                <p>Royal Flush: A, K, Q, J, 10, all same suit</p>
                <p>Straight Flush: Five cards in a sequence, same suit</p>
                <p>Four of a Kind: Four cards of the same rank</p>
                <p>Full House: Three of a kind with a pair</p>
                <p>Flush: Any five cards of the same suit</p>
                <p>Straight: Five cards in sequence</p>
                <p>Three of a Kind: Three cards of the same rank</p>
                <p>Two Pair: Two different pairs</p>
                <p>One Pair: Two cards of the same rank</p>
            </div>
        </div>
        <div id="balance" class="result"></div>
        <div>
            <h3>Player Hand</h3>
            <div id="playerHand" class="hand"></div>
        </div>
        <div>
            <h3>Dealer Hand</h3>
            <div id="dealerHand" class="hand"></div>
        </div>
        <div id="result" class="result"></div>
        <div id="error" class="error"></div>
    </div>
    <script>
        async function playGame() {
            const bet = document.getElementById("betAmount").value;
            document.getElementById("error").innerText = "";
            try {
                const response = await fetch("http://localhost:8085/api/casino/poker/play", {
                    method: "POST",
                    headers: { "Content-Type": "application/json" },
                    body: JSON.stringify({ bet: parseInt(bet) })
                });
                if (!response.ok) throw new Error("Failed to play the game");
                const data = await response.json();
                // Update balance
                document.getElementById("balance").innerText = `Balance: ${data.balance || "Not provided"}`;
                // Display hands
                displayHand("playerHand", data.playerHand);
                displayHand("dealerHand", data.dealerHand);
                // Determine the winner
                const playerScore = evaluateHand(data.playerHand);
                const dealerScore = evaluateHand(data.dealerHand);
                const resultMessage = playerScore > dealerScore ? "You Win!" : "You Lose!";
                document.getElementById("result").innerText = resultMessage;
            } catch (error) {
                document.getElementById("error").innerText = "Error: " + error.message;
            }
        }
        function displayHand(handId, hand) {
            const handContainer = document.getElementById(handId);
            handContainer.innerHTML = "";  // Clear previous hand
            hand.forEach(card => {
                const cardDiv = document.createElement("div");
                cardDiv.classList.add("card");
                // Add class for red cards
                if (card.suit === "♥" || card.suit === "♦") {
                    cardDiv.classList.add("red");
                }
                cardDiv.innerHTML = `${card.rank}<br>${card.suit}`;
                handContainer.appendChild(cardDiv);
            });
        }
        function evaluateHand(hand) {
            // Basic evaluation logic
            const ranks = hand.map(card => card.rank);
            const uniqueRanks = [...new Set(ranks)];
            // Check for pairs, three of a kind, or other hand rankings
            if (uniqueRanks.length === 2) return 7;  // Full House or Four of a Kind
            if (uniqueRanks.length === 3) return 6;  // Three of a Kind or Two Pair
            if (uniqueRanks.length === 4) return 5;  // One Pair
            return 0;  // High card
        }
    </script>
</body>
</html>
